In [102]:
import requests
import base64
import json
import joblib

import pandas as pd

Analyse my top 50 spotify tracks!

1. Create and register an App on the Spotify for Developers Dashboard. Configure this app to request my authorisation, so it can access my data
2. Grab an access token given my authorisation code. Token is used for making calls to spotify web api
3. Grab my top 50 tracks through the spotify web api
4. Analyse and Profit!

### 1.

In [49]:
# request user auth
params = {'client_id': '21ae6b1753a94a2ba76175378f2c9c31',
         'response_type': 'code',
         'redirect_uri': 'https://google.com',
         'scope': 'user-library-read user-top-read'}
r = requests.get('https://accounts.spotify.com/authorize', params=params)

In [51]:
# paste the resulting url, without `` in your web browser
r.url

'https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-library-read%2Buser-top-read%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fgoogle.com%26client_id%3D21ae6b1753a94a2ba76175378f2c9c31'

In [ ]:
# copy the response query string to here, grab the code bit, which we'll use to request an access token
https://www.google.com/?code=AQCuhK15ZqOmBUAmdykvudYufr-7qCMVdqA2H7Va4OX6MRNBRn8csTd1XdEDAET6aXhFnO_V0vejsxlbHPJjnK3W5hpUqbktDLSeNwZQTv4QHrTIPTfLjHFEbCoEnjG9ZxViIBJEoOfs5RmqOs-7TnWoXq3JtLrYCcEVLxOflbWoYOpEqA9DwuUXJFE_ZVH_LwhCw9ye0_E

### 2.

In [52]:
url = 'https://accounts.spotify.com/api/token'
headers = {}
params = {}

client_id = '21ae6b1753a94a2ba76175378f2c9c31'
client_secret = 'e8592fa50f784c1387d0c7832316b945'
message = f"{client_id}:{client_secret}"
message_bytes = message.encode('ascii')
base64_bytes = base64.urlsafe_b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

In [53]:
headers['Authorization'] = f"Basic {base64_message}"

In [56]:
data = {'grant_type': 'authorization_code',
         'code': 'AQCuhK15ZqOmBUAmdykvudYufr-7qCMVdqA2H7Va4OX6MRNBRn8csTd1XdEDAET6aXhFnO_V0vejsxlbHPJjnK3W5hpUqbktDLSeNwZQTv4QHrTIPTfLjHFEbCoEnjG9ZxViIBJEoOfs5RmqOs-7TnWoXq3JtLrYCcEVLxOflbWoYOpEqA9DwuUXJFE_ZVH_LwhCw9ye0_E',
         'redirect_uri': 'https://google.com'}
r = requests.post(url, data=data, headers=headers)

In [57]:
r.json()

{'access_token': 'BQBdIExIrJ3kSJKJz5DbchCCN4BAu5yubIHMVw2YkBoKKGjvSFmseTVxtciqrzYlXz-pLPNwU8YbjstoRdqHGDoLnPeg24tyliBXIGSkNVsgCTmR63ghBm6H1IXtXquwgKxXqk9jk26gvKUuENhKBiMj8oarnQdrA625',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQBS7CTBcXMU1k5g9XTEJPZzFtIzXJYuNn_AEvSb6HMeGHVSfwzZ96fk9Jrn__GtuHonT3gNcx7W1R1Itlrn3cammhlIHTqhbKC-wah_Cc2B8INwERMC9T8mG-VH080Kncw',
 'scope': 'user-library-read user-top-read'}

In [61]:
access_token = r.json()['access_token']

#### Quick Aside: If access token expires in the duration of the analysis

...we refresh it!

In [109]:
# refresh_token = r.json('refresh_token'). Derived from response of first request for access token
refresh_token = 'AQBS7CTBcXMU1k5g9XTEJPZzFtIzXJYuNn_AEvSb6HMeGHVSfwzZ96fk9Jrn__GtuHonT3gNcx7W1R1Itlrn3cammhlIHTqhbKC-wah_Cc2B8INwERMC9T8mG-VH080Kncw'

In [116]:
# request body parameter
data = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'grant_type': 'refresh_token',
    'refresh_token': refresh_token,
    'client_id': client_id
}

In [123]:
r = requests.post(url, data=data, headers=headers) #so we still need headers despite documentation not saying so

In [124]:
r.json()

{'access_token': 'BQCo5qaMSKP0TTwgz_9ln6zgBSwla8WxA_dHUL57rZ1NKCZMWuMB9yhQ1cpg2FVJQZJkBUKW8rGV0idfpx3MCHThlcoyGJEWnd3p8gQ2w8o7vqAZTmhk30oXRy8vZDl8GCWbx5i7Yu1YtphE52MoR8jRqt7ne5i6_nN1',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': 'user-library-read user-top-read'}

In [125]:
access_token = r.json()['access_token']

### 3. Getting my top 50 songs!

In [126]:
web_api_url = 'https://api.spotify.com/v1/me/top/tracks'
web_api_header = {
    'Authorization': f"Bearer {access_token}"
}

In [64]:
web_api_params = {
    'time_range': 'medium_term',
    'limit': '50',
}

r_top_tracks = requests.get(web_api_url, headers=web_api_header, params=web_api_params)

In [71]:
# 'items' returns an array of tracks
top_50_tracks = r_top_tracks.json()['items']

In [90]:
#dump the tracks to a persistent file
joblib.dump(top_50_tracks, 'top_50_tracks_spotify')

['top_50_tracks_spotify']

In [91]:
#load the tracks from a persistent file
top_50_tracks = joblib.load('top_50_tracks_spotify')

In [93]:
# let's investigate the first track object. Params are: https://developer.spotify.com/documentation/web-api/reference/#object-trackobject
top_50_tracks[1]

{'album': {'album_type': 'SINGLE',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1CcEgi464SWZsKY5579u7z'},
    'href': 'https://api.spotify.com/v1/artists/1CcEgi464SWZsKY5579u7z',
    'id': '1CcEgi464SWZsKY5579u7z',
    'name': 'Patrick Hernandez',
    'type': 'artist',
    'uri': 'spotify:artist:1CcEgi464SWZsKY5579u7z'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
  

There are a couple of interesting parameters already from this first query result that might influence my liking of the said tracks. Let's extract them:
* `name`
* `artists`
* `album`
* `duration_ms`
* `explicit`
* `popularity`

and let's grab each track's `id` as well, so we can look up more interesting features later on. 

We store them into lists to preserve ordering

In [99]:
# create the lists
track_id, name, artist, album, duration_ms, explicit, popularity = [], [], [], [], [], [], []

In [100]:
# populate the lists
for track in top_50_tracks:
    track_id.append(track['id'])
    name.append(track['name'])
    artist.append([artists['name'] for artists in track['artists']])
    album.append(track['album']['name'])
    duration_ms.append(track['duration_ms'])
    explicit.append(track['explicit'])
    popularity.append(track['popularity'])

In [147]:
# store all info in a data-frame
df = pd.DataFrame({'track_id': track_id,
                  'track_name': name,
                  'track_artist': artist,
                  'track_album': album,
                  'track_duration': duration_ms,
                  'track_explicit': explicit,
                  'track_popularity': popularity})

In [148]:
df.head()

,track_id,track_name,track_artist,track_album,track_duration,track_explicit,track_popularity
0,6hyQVqfHFvsJDjcmj4qrHf,Fun Tonight,[Lady Gaga],Chromatica,173426,False,65
1,3XIEWK1V9n25PS9Vb6axj5,Born to Be Alive,[Patrick Hernandez],Born to Be Alive,188133,False,70
2,2rv6y7iK48s6ocnMCjl2GK,Baba Yetu,[Stellenbosch University Choir],Heavens' flock,216872,False,37
3,6U5ySz3l4LtpaJ37pr3bGU,The Man,[The Killers],Wonderful Wonderful (Deluxe),250093,False,49
4,0y60itmpH0aPKsFiGxmtnh,Wait a Minute!,[WILLOW],ARDIPITHECUS,196520,False,83


Now that we have some basic information about my top 50 tracks in a data-frame, let's extend this information to include track-specific audio features. 

These features are stored in two objects:
* an [`AudioFeaturesObject`](https://developer.spotify.com/documentation/web-api/reference/#object-audiofeaturesobject) with interesting dimensions that are somewhat subjective, like `acousticness`, `energy`, `danceability`, `speechiness`, etc. 
* an [`AudioAnalysisObject`]() with more technical and objective song features

In [139]:
acousticness, danceability, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence = [], [], [], [], [], [], [], [], [], [], [], []

In [140]:
#grabbing audio features - might take a while
for id in track_id:
    r = requests.get(f"https://api.spotify.com/v1/audio-features/{id}", headers=web_api_header)
    track = r.json()
    acousticness.append(track['acousticness'])
    danceability.append(track['danceability'])
    energy.append(track['energy'])
    instrumentalness.append(track['instrumentalness'])
    key.append(track['key'])
    liveness.append(track['liveness'])
    loudness.append(track['loudness'])
    mode.append(track['mode'])
    speechiness.append(track['speechiness'])
    tempo.append(track['tempo'])
    time_signature.append(track['time_signature'])
    valence.append(track['valence'])

In [141]:
# lets check whethe results of api call are successful
print(len(acousticness), len(valence)) # should print 50 each

50 50


In [150]:
df['acousticness'], df['danceability'], df['energy'], df['instrumentalness'], df['key'], df['liveness'], df['loudness'], df['mode'], df['speechiness'], df['tempo'], df['time_signature'], df['valence'] = acousticness, danceability, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence

In [151]:
df.head()

,track_id,track_name,track_artist,track_album,track_duration,track_explicit,track_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,6hyQVqfHFvsJDjcmj4qrHf,Fun Tonight,[Lady Gaga],Chromatica,173426,False,65,0.3640,0.645,0.923,0.000001,11,0.1400,-4.262,1,0.0725,117.945,4,0.323
1,3XIEWK1V9n25PS9Vb6axj5,Born to Be Alive,[Patrick Hernandez],Born to Be Alive,188133,False,70,0.0846,0.704,0.867,0.000000,2,0.2060,-5.808,1,0.0315,131.467,4,0.809
2,2rv6y7iK48s6ocnMCjl2GK,Baba Yetu,[Stellenbosch University Choir],Heavens' flock,216872,False,37,0.9460,0.217,0.327,0.019300,1,0.1320,-19.916,1,0.0695,89.378,4,0.223
3,6U5ySz3l4LtpaJ37pr3bGU,The Man,[The Killers],Wonderful Wonderful (Deluxe),250093,False,49,0.0747,0.605,0.942,0.000002,11,0.0708,-2.600,0,0.1130,106.015,4,0.507
4,0y60itmpH0aPKsFiGxmtnh,Wait a Minute!,[WILLOW],ARDIPITHECUS,196520,False,83,0.0371,0.764,0.705,0.000019,3,0.0943,-5.279,0,0.0278,101.003,4,0.672


Refactor later:
* hardcode urls into the get requests instead of storing in a variable, unless appropriate to
* automatically make user sign in when you run the cell, and collect auth code
* automatically create lists to store features (instead of creating 12 lists like i did above pre-scraping) based on what features we scrape during the API calls?
* package created lists into a named dictionary for ease of adding to existing dataframe later


### 4. Analyse and Profit

In [153]:
df.describe(include='all')

,track_id,track_name,track_artist,track_album,track_duration,track_explicit,track_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,50,50,50,50,50.000000,50,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
unique,50,49,37,40,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,45aBsnKRWUzhwbcqOJLwfe,This Is Me,[The Killers],Battle Born (Deluxe Edition),NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,2,6,5,NaN,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,231075.280000,NaN,52.880000,0.239859,0.596240,0.625080,0.100210,5.140000,0.181826,-8.228120,0.760000,0.076532,117.605100,3.940000,0.396788
std,NaN,NaN,NaN,NaN,62020.157015,NaN,20.691751,0.317354,0.185468,0.232834,0.228904,3.827985,0.138545,4.473031,0.431419,0.070138,27.958179,0.469911,0.216900
min,NaN,NaN,NaN,NaN,134239.000000,NaN,3.000000,0.000254,0.113000,0.118000,0.000000,0.000000,0.042400,-21.397000,0.000000,0.022800,73.734000,1.000000,0.033900
25%,NaN,NaN,NaN,NaN,186852.000000,NaN,42.500000,0.023000,0.485250,0.457000,0.000003,1.000000,0.098775,-9.246250,1.000000,0.031950,96.447500,4.000000,0.221500
50%,NaN,NaN,NaN,NaN,217730.500000,NaN,52.000000,0.091150,0.619500,0.635500,0.000088,5.000000,0.130500,-7.185000,1.000000,0.044950,112.485500,4.000000,0.362500
75%,NaN,NaN,NaN,NaN,260602.000000,NaN,69.500000,0.336250,0.706250,0.804000,0.018975,8.750000,0.214750,-4.605500,1.000000,0.076725,132.649000,4.000000,0.568250
